In [ ]:
"""
Example Usage of the Refactored QEMU VM Manager
==============================================
This module demonstrates how to use the refactored QemuVMManager and SSHManager
classes to create and manage VM environments for AI agents.
"""

import json
import logging
import os
import time
from pathlib import Path
from typing import List

from sandbox.qemu import QemuVMManager, VMConfig, VMState

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger("VMExample")


def vm_state_change_callback(new_state: VMState) -> None:
    """
    Callback function invoked when VM state changes.

    Args:
        new_state: The new state value of the VM.
    """
    logger.info(f"VM state changed to: {new_state.value}")


def vm_error_callback(error: Exception) -> None:
    """
    Callback function invoked when a VM error occurs.

    Args:
        error: The exception instance that occurred.
    """
    logger.error(f"VM error occurred: {error}")


def create_and_configure_vm() -> None:
    """
    Create and configure a single VM environment.

    This function demonstrates:
      - Creating a default configuration file (if needed)
      - Loading the configuration from file
      - Creating and starting the container
      - Waiting for SSH availability
      - Using SSHManager to run simple commands and set up a 9p shared folder and screenshot directory
      - Reporting the container's status.
    """
    config = VMConfig(
        image="qemux/qemu",
        container_name="sandbox1",
        base_vm_dir=os.path.abspath("docker/vms/ubuntu-noble"),
        env_dir=os.path.abspath("docker/environments"),
        shared_path=os.path.abspath("docker/shared"),
        boot="ubuntu",
        ram_size="4G",
        cpu_cores="4",
        disk_size="16G",
        novnc_port=8006,
        ssh_port=2222,
        restart_policy="always",
        mount_tag="shared",
        debug="Y",
        extra_env={"DISPLAY": ":0", "SANDBOX_SCREENSHOTS_PATH": "/home/user/screenshots"},
    )

    # Create VM manager with the loaded configuration and logger
    vm_manager = QemuVMManager(config=config, logger=logger)
    vm_manager.register_callbacks(on_state_change=vm_state_change_callback, on_error=vm_error_callback)

    try:
        container = vm_manager.get_or_create_container(start_container=True)
        logger.info(f"Container started with ID: {container.id}")

        # Wait for SSH availability; increase attempts or delay if needed
        ssh_ready = vm_manager.wait_for_ssh(max_attempts=15, initial_delay=5)
        if not ssh_ready:
            logger.error("SSH is not available, aborting.")
            return

        # Create an SSH manager based on the VM manager's configuration
        ssh_manager = vm_manager.create_ssh_manager()

        # Configure 9p file sharing and create a screenshot directory
        if vm_manager.configure_9p_in_vm(ssh_manager):
            logger.info("9p file sharing configured in VM.")
        screenshot_dir = vm_manager.setup_screenshot_directory(ssh_manager, vm_id="qemu-docker-test")
        logger.info(f"Screenshot directory created at: {screenshot_dir}")

        # Use the SSH connection to execute commands
        with ssh_manager.ssh_connection():
            result = ssh_manager.exec_command("uname -a")
            logger.info(f"System info: {result['stdout'].strip()}")
            # Create a test file and check its content
            # ssh_manager.exec_command("echo 'Hello from VM' > /home/user/test.txt")
            # result = ssh_manager.exec_command("cat /home/user/test.txt")
            # logger.info(f"Test file content: {result['stdout'].strip()}")

            ssh_manager.transfer_directory("server/", "/home/user/server/")

            result = ssh_manager.exec_command("ls -l /home/user/server/")
            logger.info(f"Server directory contents: {result['stdout'].strip()}")

            ssh_manager.exec_command("cd /home/sandbox-user/server/ && bash start.sh")

        # Show container status as a JSON dump
        # status = vm_manager.get_container_status()
        # logger.info(f"Container status:\n{json.dumps(status, indent=2)}")

        logger.info("VM is running. Press Ctrl+C to stop...")
        try:
            while True:
                time.sleep(1)
        except KeyboardInterrupt:
            logger.info("Stopping VM...")

    finally:
        # Clean up resources
        vm_manager.cleanup_vm(delete_storage=False)
        logger.info("VM resources cleaned up.")


In [ ]:
# Ensure base directories exist
os.makedirs("./docker/vms/ubuntu-noble", exist_ok=True)
os.makedirs("./docker/shared", exist_ok=True)

# Uncomment one of the following:

# Single VM test:
create_and_configure_vm()

# Or create multiple VMs:
# vm_managers = create_multiple_vms(3)
# try:
#     logger.info("Multiple VMs are running. Press Ctrl+C to stop...")
#     while True:
#         time.sleep(1)
# except KeyboardInterrupt:
#     logger.info("Stopping all VMs...")
# finally:
#     for vm_manager in vm_managers:
#         vm_manager.cleanup_vm()